In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
from sklearn.model_selection import cross_val_score

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [3]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [4]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [15]:
# Group into sentences.
alice_sents = [[sent.text, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent.text, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,Alice was beginning to get very tired of sitti...,Carroll
1,So she was considering in her own mind (as wel...,Carroll
2,There was nothing so VERY remarkable in that; ...,Carroll
3,Oh dear!,Carroll
4,I shall be late!',Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [17]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    doc = doc.lower()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [18]:
corpus = sentences
corpus.columns=['sentence','label']
corpus.head()

,sentence,label
0,Alice was beginning to get very tired of sitti...,Carroll
1,So she was considering in her own mind (as wel...,Carroll
2,There was nothing so VERY remarkable in that; ...,Carroll
3,Oh dear!,Carroll
4,I shall be late!',Carroll


In [19]:
norm_corpus = normalize_corpus(corpus.sentence)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

cv =  CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [21]:
# get all unique words in the corpus
vocab = cv.get_feature_names()
# show document feature vectors
df = pd.DataFrame(cv_matrix, columns=vocab)

## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [31]:
sentences_df = pd.DataFrame()
sentences_df['text_sentence'] = sentences[0]
sentences_df['text_source'] = sentences[1]

In [32]:
word_counts = pd.concat([df, sentences_df], axis=1)

In [33]:
word_counts.head()

,15,16,1760,1784,1785,1787,1789,1791,1800,1803,...,yours,youth,youthful,zeal,zealand,zealous,zealously,zigzag,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [34]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9880877742946709

Test set score: 0.8515037593984962


In [57]:
cross_val_score(rfc, X_train, y_train, cv=5)

array([0.85758998, 0.84820031, 0.86677116, 0.86342229, 0.87127159])

Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [35]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 6723) (3190,)
Training set score: 0.9626959247648903

Test set score: 0.9046052631578947


In [58]:
cross_val_score(lr, X_train, y_train, cv=5)

array([0.87480438, 0.885759  , 0.90125392, 0.87755102, 0.89167975])

Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [36]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8501567398119122

Test set score: 0.8449248120300752


In [39]:
from sklearn.svm import SVC
model = SVC()
train = model.fit(X_train, y_train)

print('Training set score:', model.score(X_train, y_train))
print('\nTest set score:', model.score(X_test, y_test))

Training set score: 0.6824451410658308

Test set score: 0.6917293233082706


In [42]:
cross_val_score(model, X_train, y_train, cv=5)

array([0.68231612, 0.68231612, 0.68181818, 0.68288854, 0.68288854])

Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.703466666667


col_0,Austen,Carroll
row_0,,
Austen,1158,21
Carroll,535,161


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [44]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [64]:
# Clean the Emma data.
sense = gutenberg.raw('austen-sense.txt')
sense = re.sub(r'VOLUME \w+', '', sense)
sense = re.sub(r'CHAPTER \w+', '', sense)
sense = text_cleaner(sense)
print(sense[:100])

The family of Dashwood had long been settled in Sussex. Their estate was large, and their residence 


In [65]:
# Parse our cleaned data.
sense_doc = nlp(sense)

In [94]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
sense_sents = [[sent.text, "Austen"] for sent in sense_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
sense_sents = sense_sents[0:len(alice_sents)]

In [95]:
sense_sents[0][0]

'The family of Dashwood had long been settled in Sussex.'

In [96]:
sense_sents = sense_sents[0:len(alice_sents)]
sense_sentences = pd.DataFrame(sense_sents)
sense_sentences

,0,1
0,The family of Dashwood had long been settled i...,Austen
1,"Their estate was large, and their residence wa...",Austen
2,The late owner of this estate was a single man...,Austen
3,"But her death, which happened ten years before...",Austen
4,"In the society of his nephew and niece, and th...",Austen
5,His attachment to them all increased.,Austen
6,The constant attention of Mr. and Mrs. Henry D...,Austen
7,"By a former marriage, Mr. Henry Dashwood had o...",Austen
8,"The son, a steady respectable young man, was a...",Austen
9,"By his own marriage, likewise, which happened ...",Austen


In [97]:
alice_sents = [[sent.text, "Carroll"] for sent in alice_doc.sents]
# Combine the sentences from the two novels into one data frame.
alice_sentences = pd.DataFrame(alice_sents)
alice_sentences.head()

,0,1
0,Alice was beginning to get very tired of sitti...,Carroll
1,So she was considering in her own mind (as wel...,Carroll
2,There was nothing so VERY remarkable in that; ...,Carroll
3,Oh dear!,Carroll
4,I shall be late!',Carroll


In [101]:
df_corpus = pd.concat([sense_sentences,alice_sentences])
df_corpus.columns=['sentence','label']
df_corpus.head()

,sentence,label
0,The family of Dashwood had long been settled i...,Austen
1,"Their estate was large, and their residence wa...",Austen
2,The late owner of this estate was a single man...,Austen
3,"But her death, which happened ten years before...",Austen
4,"In the society of his nephew and niece, and th...",Austen


In [105]:
norm_corpus = normalize_corpus(df_corpus.sentence)

norm_corpus[:10]

array(['family dashwood long settled sussex .',
       'estate large , residence norland park , centre property , , many generations , lived respectable manner engage general good opinion surrounding acquaintance .',
       'late owner estate single man , lived advanced age , many years life , constant companion housekeeper sister .',
       'death , happened ten years , produced great alteration home ; supply loss , invited received house family nephew mr . henry dashwood , legal inheritor norland estate , person intended bequeath .',
       "society nephew niece , children , old gentleman ' days comfortably spent .",
       'attachment increased .',
       'constant attention mr . mrs . henry dashwood wishes , proceeded merely interest , goodness heart , gave every degree solid comfort age could receive ; cheerfulness children added relish existence .',
       'former marriage , mr . henry dashwood one son : present lady , three daughters .',
       'son , steady respectable young ma

In [106]:
from sklearn.feature_extraction.text import CountVectorizer

cv =  CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
vocab = cv.get_feature_names()
# show document feature vectors
df_sense = pd.DataFrame(cv_matrix, columns=vocab)

In [107]:
df_sense

,7000l,abhor,abhorred,abhorrence,abide,abilities,ability,able,ablest,abode,...,yesterday,yet,yield,young,younger,youth,youthful,zealand,zealously,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:

X_train, X_test, y_train, y_test = train_test_split(df_sense, 
                                                    df_corpus.label,
                                                    test_size=0.3,
                                                    random_state=0)

In [111]:
X_train.shape, X_test.shape

((2336, 4725), (1002, 4725))

In [112]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2336, 4725) (2336,)
Training set score: 0.9751712328767124

Test set score: 0.8902195608782435


In [113]:
lr_sense_predicted = lr.predict(X_test)
pd.crosstab(y_test, lr_sense_predicted)

col_0,Austen,Carroll
label,,
Austen,443,75
Carroll,35,449


It looks like this model is actually able to differentiate multiple works by Austen from austen-sense and Carroll from Alice in Wonderland